## Usage

- python 3.7
- pipenv
- dotenv
- twitter API (and client library)

In [23]:
# stdlib
import os
import pickle

# libraries
import twitter
from dotenv import load_dotenv

In [6]:
load_dotenv()

CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")
ACCESS_TOKEN_KEY = os.getenv("ACCESS_TOKEN_KEY")
ACCESS_TOKEN_SECRET = os.getenv("ACCESS_TOKEN_SECRET")

# WARNING: do not commit this file to git with these values printed to a cell's output

In [ ]:
api = twitter.Api(consumer_key=CONSUMER_KEY,
                  consumer_secret=CONSUMER_SECRET,
                  access_token_key=ACCESS_TOKEN_KEY,
                  access_token_secret=ACCESS_TOKEN_SECRET)
print(api.VerifyCredentials())

In [12]:
term = "libspill"
per_page = 100 # max for this endpoint

In [ ]:
#raw_query = f"q={term}%20&result_type=recent&since=2014-07-19&count={per_page}"
raw_query = f"q={term}%20&result_type=recent&count={per_page}"
#results = api.GetSearch(raw_query=raw_query)

In [17]:
len(results)
# => 100, even though i initially ran it with `per_page = 500`

type(results)
# => list

type(results[0])
# => twitter.models.Status

twitter.models.Status

In [21]:
results[0]

Status(ID=1039485380934815746, ScreenName=DoNutCentre, Created=Tue Sep 11 12:06:42 +0000 2018, Text='RT @MeckeringBoy: “Explains” is inaccurate. Verbal  diarrhoea would be more accurate. “Wall of sound Morrison” is not explaining. 💩💩💩#libsp…')

## Data Plans

- if there are no cached tweets (or if some flag is passed)
- search for tweets with a given hashtag using 30 day api in batches of 100 tweets
- receive 100 tweets, filter them by some characteristics and stream the filtered tweets into a collection
- page forward and repeat
- when the collection contains N tweets (or if we reach the end of the 30 day feed?) then cease fetching
- cache the results so we can access them from a file and not run out of monthly requests...

In [26]:
cache_filename = f"cached_tweets_{term}.pkl"
with open(cache_filename, 'wb') as f:
    pickle.dump(results, f)

In [31]:
test_load_results = None
with open(cache_filename, 'rb') as f:
    test_load_results = pickle.load(f)

In [34]:
type(test_load_results[0])

twitter.models.Status

## Ideas

- some potential hashtags include #libspill and #auspol
- filter tweets: too short, replace/ignore hyperlinks, strip formatting ('\n') etc...

## Notes

- will need to handle API rate limiting (15 calls in 15 mins)
    - one option is to use the client libraries sleep function: `sleep_on_rate_limit=True`
    - alternatively, we could catch an error or attempt to use our own loops